In [ ]:
#
# Create initial tweets database from csv
#
import csv, sqlite3

# Connect to database, drop table and recreate it if it exists
con = sqlite3.connect("db/db_tweets.db")
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS tweets;")
cur.execute("CREATE TABLE IF NOT EXISTS tweets (party, handle, tweet);")

# Extract tweets from csv
with open('original_data/ExtractedTweets.csv','rt',encoding='utf-8') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['Party'], i['Handle'], i['Tweet']) for i in dr]

# Insert extracted tweets into database table
cur.executemany("INSERT INTO tweets (party, handle, tweet) VALUES (?, ?, ?);", to_db)
con.commit()
con.close()

In [ ]:
#
# Create initial stop-words database
#
import csv, sqlite3

# Connect to database, drop table and recreate it if it exists
con = sqlite3.connect("db/db_stop-words.db")
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS stopwords;")
cur.execute("CREATE TABLE IF NOT EXISTS stopwords (word);")

# Extract stop words from csv
with open('original_data/stop-word-list.csv', newline='') as csvfile:
    dr = csv.reader(csvfile)
    to_db = []
    for i in dr:
        to_db = [(j.strip(), ) for j in i]

# Insert extracted stop words into database table
cur.executemany("INSERT INTO stopwords (word) VALUES (?);", (to_db))
con.commit()
con.close()

In [ ]:
#
# First inquiry
# Analyzing 'mudslinging': as defined by the amount of instances the parties mention some keywords
#  that are commonly used to refer to the opposing party.
#
import sqlite3
import matplotlib.pyplot as plt
import re

# Function used to count instances of words within all tweets based on regular expression provided
def get_mud(tweets, regex):
    result = 0
    for tweet in tweets:
        if(re.search(regex, tweet[0], re.I)):
            result+=1
    return result 

# Connect to database
conn = sqlite3.connect('db/db_tweets.db')
cur = conn.cursor()

# Analyze Democrat Tweets
cur.execute("SELECT tweet FROM tweets WHERE party='Democrat'")
regex = r"(?!\brepublic\b)\brepub+|\bgop\b|\bconservative+|\bright\w*wing\b"
mud_d = get_mud(cur.fetchall(), regex)

# Analyze Republican Tweets
cur.execute("SELECT tweet FROM tweets WHERE party='Republican'")
regex = r"(?!\bdemarest)(?!\bdemocracy\b)\bdem+|\bdnc\b|\bliberal+|\blibs+|\bleft\w*wing\b"
mud_r = get_mud(cur.fetchall(), regex)

# Plot results
fig, ax = plt.subplots()
ax.bar(("Democrats","Republicans"),(mud_d,mud_r),color=("lightblue","#ff6666"))
ax.set_xlabel('Political Party')
ax.set_ylabel('# of instances')
ax.set_title('Instances of political parties mentioning opposing party')
fig.tight_layout()
plt.show()

# Close connection to database
conn.close()

In [ ]:
#
# Second inquiry
# Analyzing most-used words: as defined by the strings of text between spaces within these tweets, disregarding
#  words commonly referred to as 'stop words' that are so commonly used within the English language that they hold
#  little of interest for this analysis
#
import sqlite3
import matplotlib.pyplot as plt
import re
from matplotlib_venn import venn2

# Function for creating word filter list
def filter_word(word, stopwords):
    wordFilterList = ['rt', '&amp;', '-']
    wordFilterList.extend(stopwords)
    if not word.lower().strip() in wordFilterList:
        return True

# Function for plotting bar graphs for visualizing most-used words
def plot_words(words, counts, fig_title, fig_color):
    fig, ax = plt.subplots()
    ax.barh(words,counts,color=fig_color)
    ax.set_xlabel('# of instances')
    ax.set_ylabel('Word')
    ax.set_title('Top ten words used in tweets from {}'.format(fig_title))
    fig.tight_layout()
    plt.show()

# Function for gathering most-used words from tweets
def analyze(tweets, stopwords, fig_title, fig_color, limit):
    dWords = dict()
    for tweet in tweets:
        for word in tweet[0].lower().split():
            if not filter_word(word, stopwords):
                continue
            elif word in dWords:
                dWords[word] += 1
            else:
                dWords[word] = 1
    
    d_view = [ (v,k) for k,v in dWords.items() ]
    d_view.sort(reverse=True) # Sort tuples by # of instances
    
    # Create list of most-used words and # of instances
    word_list = []
    word_count = []
    index = 0
    for v,k in d_view:
        index+=1
        if index > limit:
            break
        word_list.append(k)
        word_count.append(v)
        #print("{} - {}: {}".format(index,k,v))
    
    plot_words(word_list, word_count, fig_title, fig_color)
    return word_list

# Set number of most-used words to get
limit = 100

# Get stop words from stop-words database
stopwords = []
con = sqlite3.connect("db/db_stop-words.db")
cur = con.cursor()
cur.execute("SELECT word FROM stopwords")
for i in cur.fetchall():
    stopwords.append(i[0])
con.commit()
con.close()

# Connect to tweets database
conn = sqlite3.connect('db/db_tweets.db')
cur = conn.cursor()

# Get top words from Democrats
cur.execute("SELECT tweet FROM tweets WHERE party='Democrat'")
wList_d = analyze(cur.fetchall(), stopwords, "Democrats", "lightblue", limit)

# Get top words from Republicans
cur.execute("SELECT tweet FROM tweets WHERE party='Republican'")
wList_r = analyze(cur.fetchall(), stopwords, "Republicans", "#ff6666", limit)

# Create Venn Diagram for visualizing words in common
wSet_d = set(wList_d)
wSet_r = set(wList_r)
venn2([wSet_d, wSet_r], set_labels = ('Democrats', 'Republicans'))

# Printing most-used words
# In common
print("Words in common: ")
for w in wSet_d.intersection(wSet_r):
    print(" - {}".format(w))
# Unique to Republicans
print("\nWords unique to Republicans: ")
for w in wSet_r - wSet_d:
    print(" - {}".format(w))
# Unique to Democrats
print("\nWords unique to Democrats: ")
for w in wSet_d - wSet_r:
    print(" - {}".format(w))

# Close tweets database
conn.close()